In [11]:
import torch
import torch.nn.functional as F
import torch.utils.data
from transformers import GPT2Tokenizer, AutoConfig
from PIL import Image
from utils import proc_ques
from models.gpt import GPT2LMHeadModel
import clip_x.clip as clip
from transformers import top_k_top_p_filtering
from PIL import Image
import numpy as np
import cv2, math
import matplotlib.pyplot as plt
from captum.attr import visualization
import json
from models.gpt import NLX_GPT
from torchvision.transforms import GaussianBlur

In [12]:
caption_save_path = '/media/storage/coco/caption_data' 
annFileExp = '../self_training/cococaption/annotations/vqaX_test_annot_exp.json'
annFileFull = '../self_training/cococaption/annotations/vqaX_test_annot_full.json'
nle_data_test_path = '/media/storage/coco/VQA-X/annotated/vqaX_test.json'
nle_data_val_path = '/media/storage/coco/VQA-X/annotated/vqaX_val.json'
nle_data_train_path = '/media/storage/coco/VQA-X/annotated/vqaX_train.json'

In [13]:
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")        # load tokenizer
# model = GPT2LMHeadModel.from_pretrained(ckpt_path + model_name) # load model with config

In [5]:
def change_requires_grad(model, req_grad):
    for p in model.parameters():
        p.requires_grad = req_grad

In [6]:
clip._MODELS = {
    "ViT-B/32": "https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt",
    "ViT-B/16": "https://openaipublic.azureedge.net/clip/models/5806e77cd80f8b59890b7e101eabd078d9fb84e6937f9e85e4ecb61988df416f/ViT-B-16.pt",
    "ViT-L/14": "https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt"
}

clip_model, preprocess = clip.load("ViT-B/16", jit=False)
print("Image resolution:", clip_model.visual.input_resolution)
image_encoder = clip_model
change_requires_grad(image_encoder, False)
img_transform = preprocess

Image resolution: 224


/home/pr03/anaconda3/envs/t5/lib/python3.7/site-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [7]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
def get_elements(question_id):
    sample = data[question_id]
    img_name = sample['image_name']
    text_a = proc_ques(sample['question'])    # question

    # tokenization process
    q_segment_id, a_segment_id, e_segment_id = tokenizer.convert_tokens_to_ids(['<question>', '<answer>', '<explanation>'])
    tokens = tokenizer.tokenize(text_a)
    segment_ids = [q_segment_id] * len(tokens)

    answer = [tokenizer.bos_token] + tokenizer.tokenize(" the answer is")
    answer_len = len(answer)
    tokens += answer 

    segment_ids += [a_segment_id] * answer_len

    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = torch.tensor(input_ids, dtype=torch.long)
    segment_ids = torch.tensor(segment_ids, dtype=torch.long)

    folder = './media/storage/coco/image/train2014/' if 'train' in img_name else '/media/storage/coco/image/val2014/'   # test and val are both in val2014
    img_path = folder + img_name
    img = Image.open(img_path)
    #.convert('RGB')
    img = img_transform(img)
    qid = torch.LongTensor([int(question_id)])

    return (img, qid, input_ids, segment_ids, img_path)

# data list
data = json.load(open(nle_data_test_path, "r"))
ids_list = list(data.keys())
len(ids_list)

2127

In [8]:
config = AutoConfig.from_pretrained('distilgpt2')
config.add_cross_attention=True
model = GPT2LMHeadModel.from_pretrained("/home/pr03/jh_pjh/xAI/AAAI_2023/Self-training/ckpts/2022-07-07_11:03:06/epoch=05-val_loss=2.15.ckpt", config = config)

Some weights of the model checkpoint at /home/pr03/jh_pjh/xAI/AAAI_2023/Self-training/ckpts/2022-07-07_11:03:06/epoch=05-val_loss=2.15.ckpt were not used when initializing GPT2LMHeadModel: ['epoch', 'global_step', 'hyper_parameters', 'callbacks', 'hparams_name', 'optimizer_states', 'state_dict', 'lr_schedulers', 'pytorch-lightning_version']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at /home/pr03/jh_pjh/xAI/AAAI_2023/Self-training/ckpts/2022-07-07_11:03:06/epoch=05-val_loss=2.15.ckpt and are ne

In [9]:
from PIL import Image
import numpy as np
import cv2, math
import matplotlib.pyplot as plt

from models.gpt import NLX_GPT


load_from_epoch = 11
ckpt_path = '../self_training/ckpts/VQAX_p/'

SPECIAL_TOKENS = ['<|endoftext|>', '<pad>', '<question>', '<answer>', '<explanation>']
special_tokens_ids = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS)
because_token_id = tokenizer.convert_tokens_to_ids('Ġbecause')


nlx_gpt = NLX_GPT(visual_backbone=image_encoder, lm_backbone=model).to(device)

NameError: name 'model' is not defined

In [ ]:
def show_image_relevance(image_relevance, image, orig_image):
    # create heatmap from mask on image
    def show_cam_on_image(img, mask):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255
        cam = heatmap + np.float32(img)
        cam = cam / np.max(cam)
        return cam

    # plt.axis('off')
    # f, axarr = plt.subplots(1,2)
    # axarr[0].imshow(orig_image)

    fig, axs = plt.subplots(1, 2)
    axs[0].imshow(orig_image);
    axs[0].axis('off');
    
    feat_hw = int(math.sqrt(image_relevance.shape[-1]))
    image_relevance = image_relevance.reshape(1, 1, feat_hw, feat_hw)
    image_relevance = torch.nn.functional.interpolate(image_relevance, size=224, mode='bilinear')
    image_relevance = image_relevance.reshape(224, 224).cuda().data.cpu().numpy()
    image_relevance = (image_relevance - image_relevance.min()) / (image_relevance.max() - image_relevance.min())
    image = image[0].permute(1, 2, 0).data.cpu().numpy()
    image = (image - image.min()) / (image.max() - image.min())
    vis = show_cam_on_image(image, image_relevance)
    vis = np.uint8(255 * vis)
    vis = cv2.cvtColor(np.array(vis), cv2.COLOR_RGB2BGR)
    # axar[1].imshow(vis)
    axs[1].imshow(vis);
    axs[1].axis('off');
    # plt.imshow(vis)

In [ ]:
img_size = 224
max_seq_len = 40
do_sample = False
top_k =  0
top_p =  0.9
temperature = 1
start_layer = 11

In [ ]:
q_id = ids_list[0]
batch = get_elements(q_id)
img_path = batch[-1]
batch = tuple(input_tensor.unsqueeze(0).to(device) for input_tensor in batch[:-1])
img, img_id, input_ids, segment_ids = batch
batch_size = img.shape[0]

In [10]:
batch[0].shape

NameError: name 'batch' is not defined

In [102]:
img_relevance_maps = []
current_output = []
current_logits = []
always_exp = False

In [111]:
logits = nlx_gpt(image=img, input_ids=input_ids, segment_ids=segment_ids)
logits.shape

torch.Size([1, 12, 7, 7])
torch.Size([1, 12, 7, 196])
torch.Size([1, 12, 7, 7])
torch.Size([1, 12, 7, 196])
torch.Size([1, 12, 7, 7])
torch.Size([1, 12, 7, 196])
torch.Size([1, 12, 7, 7])
torch.Size([1, 12, 7, 196])
torch.Size([1, 12, 7, 7])
torch.Size([1, 12, 7, 196])
torch.Size([1, 12, 7, 7])
torch.Size([1, 12, 7, 196])


torch.Size([1, 50257])

In [17]:
image_attn_blocks = list(dict(nlx_gpt.visual_encoder.visual.transformer.resblocks.named_children()).values())
num_tokens = image_attn_blocks[0].attn_probs.shape[-1]

In [18]:
logits.shape

torch.Size([1, 50257])

In [106]:
probas = logits.softmax(-1)[0, :, :-1]
probas.shape

IndexError: too many indices for tensor of dimension 2

In [104]:
from transformers import top_k_top_p_filtering
filtered_logits = top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p)
filtered_logits.shape

torch.Size([1, 50257])

In [105]:
probs = F.softmax(filtered_logits, dim=-1)
probs.shape

torch.Size([1, 50257])

In [110]:
prev2 = torch.multinomial(probs,7)
prev2

tensor([[24829, 13965, 10130, 21311,  6787, 20478, 27518]], device='cuda:0')

In [21]:
prev = torch.multinomial(probs, dim=-1) if do_sample else torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
prev

tensor([[20457]], device='cuda:0')

In [22]:
one_hot = F.one_hot(prev, num_classes=logits.shape[-1]).type(torch.float32)
one_hot

tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')

In [23]:
one_hot = one_hot.requires_grad_(True).cuda()

In [24]:
one_hot = torch.sum(one_hot.cuda()*logits, dim=-1)
one_hot

tensor([[2.3458]], device='cuda:0', grad_fn=<SumBackward1>)

In [25]:
nlx_gpt.zero_grad()

In [30]:
grad = torch.autograd.grad(one_hot, [image_attn_blocks[11].attn_probs], retain_graph=True)[0].detach()
grad.shape

torch.Size([12, 197, 197])

In [30]:
for i, blk in enumerate(image_attn_blocks):
        if i < start_layer:
            continue
        grad = torch.autograd.grad(one_hot, [blk.attn_probs], retain_graph=True)[0].detach()
        cam = blk.attn_probs.detach()
        cam = cam.reshape(-1, cam.shape[-1], cam.shape[-1])
        grad = grad.reshape(-1, grad.shape[-1], grad.shape[-1])
        cam = grad * cam
        cam = cam.reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
        cam = cam.clamp(min=0).mean(dim=1)
        R = R + torch.bmm(cam, R)

In [71]:
encoder_blocks = list(dict(nlx_gpt.visual_encoder.visual.transformer.resblocks.named_children()).values())
decoder_blocks = nlx_gpt.lm.transformer.h

In [86]:
image_bboxes = encoder_blocks[0].attn_probs.shape[-1]
queries_num = decoder_blocks[0].attn.get_attn().shape[-1]

# image self attention matrix
R_i_i = torch.eye(image_bboxes, image_bboxes).to(encoder_blocks[0].attn_probs.device)
# queries self attention matrix
R_q_q = torch.eye(queries_num, queries_num).to(encoder_blocks[0].attn_probs.device)
# impact of image boxes on queries
R_q_i = torch.zeros(queries_num, image_bboxes-1).to(encoder_blocks[0].attn_probs.device)

In [87]:
encoder_blocks[0].attn_probs.shape

torch.Size([12, 197, 197])

In [88]:
def avg_heads(cam, grad):
    cam = cam.reshape(-1, cam.shape[-2], cam.shape[-1])
    grad = grad.reshape(-1, grad.shape[-2], grad.shape[-1])
    cam = grad * cam
    cam = cam.clamp(min=0).mean(dim=0)
    return cam

In [89]:
for blk in encoder_blocks:
    grad = torch.autograd.grad(one_hot, [blk.attn_probs], retain_graph=True)[0].detach()
    cam = blk.attn_probs.detach()
    cam = avg_heads(cam, grad)
    R_i_i += torch.matmul(cam, R_i_i)

In [91]:
def apply_self_attention_rules(R_ss, R_sq, cam_ss):
    R_sq_addition = torch.matmul(cam_ss, R_sq)
    R_ss_addition = torch.matmul(cam_ss, R_ss)
    return R_ss_addition, R_sq_addition

In [92]:
for blk in decoder_blocks:
    grad = torch.autograd.grad(one_hot, [blk.attn.get_attn()], retain_graph=True)[0].detach()
    cam = blk.attn.get_attn().detach()
    cam = avg_heads(cam, grad)
    R_q_q_add, R_q_i_add = apply_self_attention_rules(R_q_q, R_q_i, cam)
    R_q_q += R_q_q_add
    R_q_i += R_q_i_add

In [93]:
def handle_residual(orig_self_attention):
    self_attention = orig_self_attention.clone()
    diag_idx = range(self_attention.shape[-1])
    self_attention -= torch.eye(self_attention.shape[-1]).to(self_attention.device)
    assert self_attention[diag_idx, diag_idx].min() >= 0
    self_attention = self_attention / self_attention.sum(dim=-1, keepdim=True)
    self_attention += torch.eye(self_attention.shape[-1]).to(self_attention.device)
    return self_attention

In [94]:
def apply_mm_attention_rules(R_ss, R_qq, cam_sq, apply_normalization=True, apply_self_in_rule_10=True):
    R_ss_normalized = R_ss
    R_qq_normalized = R_qq
    if apply_normalization:
        R_ss_normalized = handle_residual(R_ss)
        R_qq_normalized = handle_residual(R_qq)
    R_sq_addition = torch.matmul(R_ss_normalized.t(), torch.matmul(cam_sq, R_qq_normalized))
    if not apply_self_in_rule_10:
        R_sq_addition = cam_sq
    R_sq_addition[torch.isnan(R_sq_addition)] = 0
    
    return R_sq_addition

In [95]:
for blk in decoder_blocks:
    grad_q_i = torch.autograd.grad(one_hot, [blk.crossattention.get_attn()], retain_graph=True)[0].detach()
    cam_q_i = blk.crossattention.get_attn().detach()
    cam_q_i = avg_heads(cam_q_i, grad_q_i)
    R_q_i += apply_mm_attention_rules(R_q_q, R_i_i[1:,1:], cam_q_i,apply_normalization=True,apply_self_in_rule_10=True)


In [96]:
aggregated = R_q_i.unsqueeze_(0)
aggregated.shape

torch.Size([1, 7, 196])

In [ ]:
aggregated = aggregated[:,target_index, :].unsqueeze_(0).detach()
